# **Installing Necessary Libraries**

In this cell, we are installing the required Python libraries for our data extraction and analysis tasks. These libraries include BeautifulSoup for web scraping, requests for handling HTTP requests, nltk for natural language processing, pandas for data manipulation, and openpyxl for working with Excel files. Run the following command to install these dependencies:

In [1]:
# Install the necessary libraries
!pip install beautifulsoup4 requests nltk pandas openpyxl

# **Extracting Articles from URLs**

Here, we import the necessary libraries and define the functions for extracting article text from the given URLs listed in an Excel file.

**Steps:**

* Import Libraries: Import the required libraries for HTTP requests,
HTML parsing, and data handling.
* Load URLs: Read the URLs from the input Excel file using pandas.
* Create Directory: Ensure a directory exists for storing the extracted articles.
* Fetch Articles: Define a function to fetch and parse the article content from each URL.
* Save Articles: Iterate through the URLs, fetch the article text, and save them to text files in the designated directory.

The articles are saved in the articles directory with filenames based on their URL IDs.

In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font

# Load the URLs from the input Excel file
input_file = '/content/Input.xlsx'
df = pd.read_excel(input_file)

# Create a directory for the extracted articles
os.makedirs('articles', exist_ok=True)

# Function to fetch and extract article text
def fetch_article_text(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        # Extract title and article text
        title = soup.find('title').get_text()
        article_text = ' '.join(p.get_text() for p in soup.find_all('p'))
        return title, article_text
    return None, None

# Iterate over the URLs and save the articles
for idx, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    title, article_text = fetch_article_text(url)
    if title and article_text:
        with open(f'articles/{url_id}.txt', 'w', encoding='utf-8') as file:
            file.write(f"{title}\n\n{article_text}")

print("Articles have been successfully extracted.")

Articles have been successfully extracted.


# **Text Analysis and Calculation of Variables**

This cell is responsible for loading the extracted articles, performing text analysis, and calculating various linguistic metrics.

**Steps:**

* Load Stop Words: Read stop words from multiple text files and combine them into a single set.
* Load Positive and Negative Words: Load lists of positive and negative words.
* Define Text Cleaning Function: A function to clean and tokenize the text, removing stop words and non-alphabetic tokens.
* Calculate Variables: Define a function to compute various text analysis metrics such as word count, sentence count, positivity/negativity scores, fog index, and more.
* Process Articles: Iterate through the saved article files, calculate the defined variables, and store the results.
Merge Results with Output Structure: Merge the calculated results with a predefined output structure from an Excel file.
* Save Results: Save the updated output to a new Excel file and format the URLs as clickable hyperlinks.

The output is saved as 'Output Data Structure Updated.xlsx'.

In [26]:
# Load stop words from text files
stop_words = set()
stop_word_files = ['/content/StopWords_Auditor.txt', '/content/StopWords_Currencies.txt', '/content/StopWords_DatesandNumbers.txt', '/content/StopWords_Generic.txt', '/content/StopWords_GenericLong.txt', '/content/StopWords_Geographic.txt', '/content/StopWords_Names.txt']

for file_path in stop_word_files:
    with open(file_path, 'r', encoding='latin-1') as file:
        stop_words.update(file.read().split())

# Load positive and negative words
with open('/content/positive-words.txt', 'r', encoding='latin-1') as file:
    positive_words = set(file.read().split())

with open('/content/negative-words.txt', 'r', encoding='latin-1') as file:
    negative_words = set(file.read().split())

# Function to clean and tokenize text
def clean_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]
    return tokens

# Function to calculate variables
def calculate_variables(text):
    tokens = clean_text(text)
    sentences = sent_tokenize(text)
    word_count = len(tokens)
    sentence_count = len(sentences)
    complex_words = [word for word in tokens if len(word) > 2]  # Simplified condition for complex words
    complex_word_count = len(complex_words)
    syllables_per_word = sum(len([char for char in word if char in 'aeiou']) for word in tokens) / word_count

    positive_score = sum(1 for word in tokens if word in positive_words)
    negative_score = sum(1 for word in tokens if word in negative_words)
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (word_count + 0.000001)
    avg_sentence_length = word_count / sentence_count
    percentage_of_complex_words = complex_word_count / word_count
    fog_index = 0.4 * (avg_sentence_length + percentage_of_complex_words)
    avg_words_per_sentence = word_count / sentence_count
    personal_pronouns = sum(1 for word in tokens if word in ["i", "we", "my", "ours", "us"])
    avg_word_length = sum(len(word) for word in tokens) / word_count

    return {
        'POSITIVE SCORE': positive_score,
        'NEGATIVE SCORE': negative_score,
        'POLARITY SCORE': polarity_score,
        'SUBJECTIVITY SCORE': subjectivity_score,
        'AVG SENTENCE LENGTH': avg_sentence_length,
        'PERCENTAGE OF COMPLEX WORDS': percentage_of_complex_words,
        'FOG INDEX': fog_index,
        'AVG NUMBER OF WORDS PER SENTENCE': avg_words_per_sentence,
        'COMPLEX WORD COUNT': complex_word_count,
        'WORD COUNT': word_count,
        'SYLLABLE PER WORD': syllables_per_word,
        'PERSONAL PRONOUNS': personal_pronouns,
        'AVG WORD LENGTH': avg_word_length
    }

# Load the extracted articles and calculate the variables
results = []
article_dir = 'articles'
for file_name in os.listdir(article_dir):
    if file_name.endswith('.txt'):
        url_id = file_name.replace('.txt', '')
        with open(os.path.join(article_dir, file_name), 'r', encoding='utf-8') as file:
            text = file.read()
            title, article_text = text.split('\n\n', 1)
            variables = calculate_variables(article_text)
            result = {
                'URL_ID': url_id,
                'URL': df[df['URL_ID'] == url_id]['URL'].values[0]
            }
            result.update(variables)
            results.append(result)

# Load the predefined output structure
output_structure_file = '/content/Output Data Structure.xlsx'
output_df = pd.read_excel(output_structure_file)

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Merge the results with the output structure
final_df = pd.merge(output_df, results_df, on=['URL_ID', 'URL'], how='left')

# Save the updated output file
final_df.to_excel('/content/Output Data Structure Updated.xlsx', index=False)

# Save the updated output file with clickable URLs
writer = pd.ExcelWriter('/content/Output Data Structure Updated.xlsx', engine='openpyxl')
final_df.to_excel(writer, index=False)

workbook = writer.book
worksheet = writer.sheets['Sheet1']

# Convert URLs into clickable hyperlinks
for idx, row in final_df.iterrows():
    url_cell = worksheet.cell(row=idx+2, column=2)  # URL column is the 2nd column
    url = row['URL']
    url_cell.hyperlink = url
    url_cell.font = Font(color="0000FF", underline="single")

writer.close()

print("Textual analysis has been completed and saved.")

Textual analysis has been completed and saved.


# **Download the Updated Output File**
This final cell provides functionality to download the updated Excel file containing the results of the text analysis.

Steps:

* Import Colab Files Module: Import the module required for file download.
* Download File: Execute the download function to allow the user to download the updated Excel file.
* I manually adjusted minor features of the excel sheet like column width,Null values etc.

In [27]:
from google.colab import files

files.download('/content/Output Data Structure Updated.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Dependencies Required**
* beautifulsoup4
* requests
* nltk
* pandas
* openpyxl